In [1]:
import numpy as np
from scipy.optimize import brentq, minimize
from scipy.integrate import quad

print("""
╔══════════════════════════════════════════════════════════════════════╗
║        OJ 287: WILL RG ORBITAL ANALYSIS AND FLARE TIMING           ║
╠══════════════════════════════════════════════════════════════════════╣
║  All equations explicitly documented for reproducibility            ║
╚══════════════════════════════════════════════════════════════════════╝
""")

# ============================================================
# SECTION 1: OBSERVATIONAL INPUT DATA
# ============================================================
print("="*70)
print("SECTION 1: OBSERVATIONAL INPUT DATA")
print("="*70)

# Historical flare epochs (from Valtonen et al. 2008, 2016, Dey et al. 2018)
# These are the OBSERVED optical outburst peaks
flare_obs = {
    # Double-peak structure: each orbit produces ~2 flares
    # when secondary crosses disk going "up" and "down"
    1912.98: "historical",
    1947.28: "historical",
    1957.08: "historical",
    1972.94: "historical",
    1982.96: "historical",
    1984.12: "historical",
    1995.84: "observed",
    2005.74: "observed",
    2007.69: "observed",
    2015.87: "observed (predicted to 2 weeks)",
    2019.57: "observed (predicted to 4 hours)",
    2022.70: "NOT OBSERVED (Komossa et al. 2023)",
}

print("\nObserved flare epochs (decimal years):")
for epoch, note in flare_obs.items():
    marker = "✗" if "NOT" in note else "✓"
    print(f"  {marker} {epoch:.2f}  ({note})")

# Key measured quantities
T_observed = 12.06          # yr - mean observed flare period
e_observed = 0.66           # eccentricity (from flare pattern fitting)
prec_observed_deg = 39.1    # degrees/orbit (Lehto & Valtonen 1996)

print(f"\nKey observational parameters:")
print(f"  T_obs  = {T_observed} yr (mean flare recurrence)")
print(f"  e_obs  = {e_observed}")
print(f"  Δφ_obs = {prec_observed_deg}°/orbit")

# Physical constants
G    = 6.67430e-11     # m³/(kg·s²)
c    = 2.99792458e8    # m/s
M_sun = 1.98892e30     # kg
yr   = 365.25 * 86400  # seconds
AU   = 1.49598e11      # m
pc   = 3.08568e16      # m

# ============================================================
# SECTION 2: WILL RG EQUATIONS (REFERENCE)
# ============================================================
print(f"\n{'='*70}")
print("SECTION 2: WILL RG EQUATIONS USED")
print("="*70)

print("""
FUNDAMENTAL RELATIONS:
  κ² = R_s/r                                              ... (W1)
  β² = v²/c²                                              ... (W2)

GLOBAL PARAMETERS (from κ_p and e):
  κ²   = κ_p²(1-e)                                        ... (W3)
  β²   = κ²/2                                             ... (W4)
  Q²   = κ² + β² = (3/2)κ²                                ... (W5)
  W    = β²/2 = κ_p²(1-e)/4                               ... (W6)
  β_p² = κ_p²(1+e)/2                                      ... (W7)
  a    = R_s/κ²                                            ... (W8)
  r_p  = a(1-e) = R_s/κ_p²                                ... (W9)

PRECESSION:
  Δφ_WILL = (3π/2)·κ_p⁴/β_p² = 2πQ²/(1-e²) = 3πκ_p²/(1+e) ... (W10)

PERIOD:
  ω = βc/a                                                 ... (W11)
  T = 2π/ω = 2πa/(βc)                                     ... (W12)

MASS:
  R_s = 2GM/(c²)                                           ... (W13)
  M   = κ_p²·c²·r_p/(2G) = κ²·c²·a/(2G)                  ... (W14)

PHASE VARIABLES:
  κ_o² = κ_p²·(1 + e·cos(o))/(1+e)                        ... (W15)
  β_o² = κ_o² - 2W                                         ... (W16)
  r_o  = R_s/κ_o²                                          ... (W17)

TIME INTEGRATION:
  ω_o(o) = (βc/a)·(1 + e·cos(o))²/(1-e²)^(3/2)           ... (W18)
  dt = do/ω_o(o)                                            ... (W19)

COORDINATE ANGLE vs ROM PHASE:
  φ(o) = o · (2π + Δφ_WILL)/(2π)                           ... (W20)
  (linear rescaling: ROM phase o ∈ [0,2π] maps to
   coordinate angle φ ∈ [0, 2π+Δφ])
""")

# ============================================================
# SECTION 3: DERIVE WILL PARAMETERS FROM OBSERVATIONS
# ============================================================
print("="*70)
print("SECTION 3: DERIVE WILL PARAMETERS")
print("="*70)

print("""
STRATEGY:
  Given: T_obs, e_obs, Δφ_obs

  Step 1: From Δφ and e, get κ_p²     via (W10)
  Step 2: From κ_p² and e, get κ², β   via (W3,W4)
  Step 3: From T and β, get a          via (W12)
  Step 4: From a and κ², get R_s       via (W8)
  Step 5: From R_s, get M              via (W13)
""")

prec_obs_rad = np.radians(prec_observed_deg)

# Step 1: κ_p² from precession (W10)
# Δφ = 3πκ_p²/(1+e) → κ_p² = Δφ(1+e)/(3π)
kp2_will = prec_obs_rad * (1 + e_observed) / (3 * np.pi)
kp_will = np.sqrt(kp2_will)

print(f"Step 1: κ_p² = Δφ_obs·(1+e)/(3π)")
print(f"  κ_p² = {prec_obs_rad:.8f} × {1+e_observed:.4f} / (3π)")
print(f"  κ_p² = {kp2_will:.10f}")
print(f"  κ_p  = {kp_will:.10f}")

# Step 2: Global parameters (W3-W7)
kappa2_will = kp2_will * (1 - e_observed)
kappa_will = np.sqrt(kappa2_will)
beta2_will = kappa2_will / 2
beta_will = np.sqrt(beta2_will)
Q2_will = kappa2_will + beta2_will
Q_will = np.sqrt(Q2_will)
W_will = beta2_will / 2
bp2_will = kp2_will * (1 + e_observed) / 2
bp_will = np.sqrt(bp2_will)

print(f"\nStep 2: Global ROM parameters")
print(f"  κ²   = κ_p²(1-e)     = {kappa2_will:.10e}")
print(f"  κ    = {kappa_will:.10f}")
print(f"  β²   = κ²/2          = {beta2_will:.10e}")
print(f"  β    = {beta_will:.10f}")
print(f"  Q²   = (3/2)κ²       = {Q2_will:.10e}")
print(f"  Q    = {Q_will:.10f}")
print(f"  W    = β²/2          = {W_will:.10e}")
print(f"  β_p  = √(κ_p²(1+e)/2) = {bp_will:.10f}")

# Step 3: Semi-major axis from period (W12)
# T = 2πa/(βc) → a = Tβc/(2π)
T_sec = T_observed * yr
a_will = T_sec * beta_will * c / (2 * np.pi)

print(f"\nStep 3: Semi-major axis from period")
print(f"  T    = {T_observed} yr = {T_sec:.6e} s")
print(f"  a    = T·β·c/(2π) = {a_will:.6e} m")
print(f"       = {a_will/AU:.2f} AU")
print(f"       = {a_will/pc:.6f} pc")

# Step 4: Schwarzschild radius (W8)
Rs_will = kappa2_will * a_will

print(f"\nStep 4: Schwarzschild radius")
print(f"  R_s  = κ²·a = {Rs_will:.6e} m")
print(f"       = {Rs_will/AU:.4f} AU")

# Step 5: Mass (W13)
M_will = Rs_will * c**2 / (2 * G)
M_will_solar = M_will / M_sun

print(f"\nStep 5: Central mass")
print(f"  M    = R_s·c²/(2G) = {M_will:.6e} kg")
print(f"       = {M_will_solar:.4e} M☉")

# Derived quantities
r_p_will = a_will * (1 - e_observed)
r_a_will = a_will * (1 + e_observed)

print(f"\nDerived orbital geometry:")
print(f"  r_p  = a(1-e) = {r_p_will:.6e} m = {r_p_will/AU:.1f} AU = {r_p_will/Rs_will:.1f} R_s")
print(f"  r_a  = a(1+e) = {r_a_will:.6e} m = {r_a_will/AU:.1f} AU")

# Verify self-consistency
kp2_check = Rs_will / r_p_will
prec_check = 3*np.pi*kp2_check/(1+e_observed)
print(f"\nSelf-consistency checks:")
print(f"  κ_p² = R_s/r_p = {kp2_check:.10f}  (input: {kp2_will:.10f})  ✓")
print(f"  Δφ   = 3πκ_p²/(1+e) = {np.degrees(prec_check):.4f}°  (input: {prec_observed_deg}°)  ✓")

# ============================================================
# SECTION 4: COMPARE WILL vs VALTONEN (GR) PARAMETERS
# ============================================================
print(f"\n{'='*70}")
print("SECTION 4: WILL vs VALTONEN (GR) PARAMETERS")
print("="*70)

# Valtonen model parameters (from literature)
M_valt = 1.835e10      # M☉
a_valt_pc = 0.06        # pc (Valtonen+2019)
a_valt = a_valt_pc * pc
Rs_valt = 2*G*M_valt*M_sun/c**2
rp_valt = a_valt*(1-e_observed)
kp2_valt = Rs_valt/rp_valt
kp_valt = np.sqrt(kp2_valt)

print(f"\n{'Parameter':>20s}  {'WILL':>16s}  {'Valtonen(GR)':>16s}  {'Δ%':>8s}")
print("-"*66)
print(f"{'M (10¹⁰ M☉)':>20s}  {M_will_solar/1e10:16.4f}  {M_valt/1e10:16.4f}  {abs(M_will_solar-M_valt)/M_valt*100:7.1f}%")
print(f"{'a (pc)':>20s}  {a_will/pc:16.6f}  {a_valt_pc:16.6f}  {abs(a_will/pc-a_valt_pc)/a_valt_pc*100:7.1f}%")
print(f"{'R_s (AU)':>20s}  {Rs_will/AU:16.4f}  {Rs_valt/AU:16.4f}  {abs(Rs_will-Rs_valt)/Rs_valt*100:7.1f}%")
print(f"{'r_p (AU)':>20s}  {r_p_will/AU:16.1f}  {rp_valt/AU:16.1f}  {abs(r_p_will-rp_valt)/rp_valt*100:7.1f}%")
print(f"{'κ_p':>20s}  {kp_will:16.8f}  {kp_valt:16.8f}  {abs(kp_will-kp_valt)/kp_valt*100:7.1f}%")
print(f"{'r_p/R_s':>20s}  {r_p_will/Rs_will:16.1f}  {rp_valt/Rs_valt:16.1f}")
print(f"{'β_p (v_p/c)':>20s}  {bp_will:16.8f}  {np.sqrt(kp2_valt*(1+e_observed)/2):16.8f}")

# ============================================================
# SECTION 5: FLARE TIMING — THE KEPLERIAN SEQUENCE
# ============================================================
print(f"\n{'='*70}")
print("SECTION 5: FLARE TIMING COMPUTATION")
print("="*70)

print("""
PHYSICAL MODEL (Lehto & Valtonen 1996):
  The secondary BH orbits in a plane inclined to the primary's
  accretion disk. It crosses the disk TWICE per orbit.
  Each crossing produces an optical flare (with a delay of days-weeks).

  The orbit precesses, so the crossing points shift each cycle.

MATHEMATICAL MODEL:
  Disk crossings occur when the secondary passes through a fixed
  plane (z = 0). In a precessing orbit, this happens when the
  orbital phase satisfies:

  φ_cross = n·π  (for crossing number n = 0, 1, 2, ...)

  where φ is the TOTAL angle from some reference direction.

  With precession, the ROM phase o at crossing is:

  o_cross = φ_cross · 2π/(2π + Δφ)                         ... (F1)

  The time of each crossing is found by integrating (W18):

  t(o) = ∫₀ᵒ do'/ω_o(o')                                   ... (F2)

  where ω_o(o) = (βc/a)·(1+e·cos(o))²/(1-e²)^(3/2)        ... (W18)
""")

def compute_time_from_phase(o_target, e, beta, a):
    """
    Integrate ROM time equation (W18,W19) from o=0 to o=o_target.
    Returns time in seconds.

    dt/do = 1/ω_o = (a/(βc)) · (1-e²)^(3/2) / (1+e·cos(o))²
    """
    prefactor = a / (beta * c) * (1 - e**2)**1.5

    def integrand(o):
        return prefactor / (1 + e * np.cos(o))**2

    result, _ = quad(integrand, 0, o_target)
    return result

# Compute crossing times
# Total coordinate angle per orbit: 2π + Δφ
Delta_phi_will = 2 * np.pi + prec_obs_rad
# Scale factor: ROM phase per coordinate radian
scale = 2 * np.pi / Delta_phi_will

print(f"Δφ_total = 2π + {prec_observed_deg:.1f}° = {np.degrees(Delta_phi_will):.2f}° = {Delta_phi_will:.6f} rad")
print(f"Scale: o = φ × {scale:.6f}")
print(f"")

# Disk crossings at φ = 0, π, 2π, 3π, ...
# Phase o at each crossing: o_n = n·π·scale
# But o must wrap around: within one orbit, o ∈ [0, 2π)
# So we need o mod 2π for the time integration

# Let's compute several orbits of crossing times
n_orbits = 12
T_orbit_will = compute_time_from_phase(2*np.pi, e_observed, beta_will, a_will)
T_orbit_yr = T_orbit_will / yr

print(f"Computed orbital period: T = {T_orbit_yr:.4f} yr  (observed: {T_observed} yr)  ✓")
print(f"\nNote: Two disk crossings per orbit (ascending and descending).")

# Reference epoch: perihelion passage
# We'll set t=0 at some perihelion, then compute crossing times
# The initial orientation angle between line of nodes and perihelion = ω₀

# The crossing condition in the precessing frame:
# The argument of the secondary w.r.t. disk plane:
# θ(t) = ω₀ + φ(t)  where φ is the total angle traveled
# Crossing when θ = n·π
#
# In ROM phase: disk crossing when the TOTAL angle
# o × (2π+Δφ)/(2π) + ω₀ = n·π
# i.e. o_cross = (n·π - ω₀) × 2π/(2π+Δφ)

# ω₀ is the initial argument of perihelion w.r.t. disk plane
# This is a FREE PARAMETER that we need to fit

# Let's compute crossing times as function of ω₀
def compute_all_crossings(omega0_deg, e, beta, a, n_orbits=12):
    """
    Compute disk crossing times for n_orbits.
    omega0 = initial angle between perihelion direction and disk line of nodes

    Returns list of (epoch_yr, crossing_number, rom_phase, orbit_number)
    """
    omega0 = np.radians(omega0_deg)
    Delta_phi = 2*np.pi + 3*np.pi*kp2_will/(1+e)  # WILL precession
    scale = 2*np.pi / Delta_phi
    T_orb = compute_time_from_phase(2*np.pi, e, beta, a)

    crossings = []

    for orbit in range(n_orbits):
        for k in range(2):  # 2 crossings per orbit
            # Total crossing number
            n = 2*orbit + k

            # Total coordinate angle at crossing
            phi_cross = n * np.pi - omega0
            if phi_cross < 0:
                continue

            # Which orbit does this fall in?
            # Total ROM phase
            o_total = phi_cross * scale
            orbit_num = int(o_total / (2*np.pi))
            o_in_orbit = o_total - orbit_num * 2*np.pi

            # Time within this orbit
            if o_in_orbit < 0 or o_in_orbit > 2*np.pi:
                continue

            t_in_orbit = compute_time_from_phase(o_in_orbit, e, beta, a)
            t_total = orbit_num * T_orb + t_in_orbit
            epoch_yr = t_total / yr

            crossings.append({
                'epoch_yr': epoch_yr,
                'n': n,
                'orbit': orbit_num,
                'o_phase': o_in_orbit,
                'r_cross': a*(1-e**2)/(1+e*np.cos(o_in_orbit))
            })

    return crossings

# ============================================================
# SECTION 6: FIT ω₀ AND t₀ TO OBSERVED FLARES
# ============================================================
print(f"\n{'='*70}")
print("SECTION 6: FIT REFERENCE EPOCH AND ORIENTATION")
print("="*70)

print("""
FREE PARAMETERS:
  t₀ : reference epoch (perihelion time)
  ω₀ : argument of perihelion relative to disk line of nodes

We minimize Σᵢ (t_predicted - t_observed)² over (t₀, ω₀).
Each observed flare is matched to the nearest predicted crossing.
""")

obs_epochs = np.array([1912.98, 1947.28, 1957.08, 1972.94, 1982.96,
                        1984.12, 1995.84, 2005.74, 2007.69, 2015.87, 2019.57])
# Exclude 2022.70 (not observed)

def cost_function(params):
    """Compute sum of squared residuals between predicted and observed flares"""
    t0, omega0 = params

    crossings = compute_all_crossings(omega0, e_observed, beta_will, a_will, n_orbits=15)
    pred_epochs = np.array([t0 + cr['epoch_yr'] for cr in crossings])

    total_cost = 0
    for obs in obs_epochs:
        if len(pred_epochs) == 0:
            return 1e10
        diffs = np.abs(pred_epochs - obs)
        total_cost += np.min(diffs)**2

    return total_cost

# Grid search over ω₀, then refine
best_cost = 1e10
best_params = None

for omega0_trial in np.linspace(0, 360, 73):  # 5° steps
    for t0_trial in np.linspace(1885, 1915, 61):  # 0.5 yr steps
        cost = cost_function([t0_trial, omega0_trial])
        if cost < best_cost:
            best_cost = cost
            best_params = [t0_trial, omega0_trial]

# Refine with optimizer
from scipy.optimize import minimize
result = minimize(cost_function, best_params, method='Nelder-Mead',
                  options={'xatol': 0.001, 'fatol': 1e-6, 'maxiter': 10000})

t0_fit, omega0_fit = result.x
print(f"Best fit parameters:")
print(f"  t₀  = {t0_fit:.3f} (perihelion reference epoch)")
print(f"  ω₀  = {omega0_fit:.2f}° (argument of perihelion vs disk)")
print(f"  RMS = {np.sqrt(result.fun/len(obs_epochs)):.3f} yr")

# ============================================================
# SECTION 7: PREDICTED vs OBSERVED FLARE TIMES
# ============================================================
print(f"\n{'='*70}")
print("SECTION 7: WILL PREDICTED vs OBSERVED FLARE EPOCHS")
print("="*70)

crossings = compute_all_crossings(omega0_fit, e_observed, beta_will, a_will, n_orbits=15)
pred_epochs = [(t0_fit + cr['epoch_yr'], cr) for cr in crossings]

print(f"\n{'Observed':>10s}  {'WILL pred':>10s}  {'Δt (yr)':>8s}  {'Δt (days)':>10s}  {'Phase':>8s}  {'r/R_s':>8s}")
print("-"*62)

matched_pred = []
for obs in sorted(list(flare_obs.keys())):
    note = flare_obs[obs]
    best_match = None
    best_dt = 1e10
    for pred_ep, cr in pred_epochs:
        dt = pred_ep - obs
        if abs(dt) < abs(best_dt):
            best_dt = dt
            best_match = (pred_ep, cr)

    if best_match:
        pred_ep, cr = best_match
        marker = "✗" if "NOT" in note else " "
        print(f"{obs:10.2f}  {pred_ep:10.2f}  {best_dt:+8.3f}  {best_dt*365.25:+10.1f}  "
              f"{np.degrees(cr['o_phase']):7.1f}°  {cr['r_cross']/Rs_will:7.1f}  {marker}")
        matched_pred.append((obs, pred_ep, best_dt))

residuals = [m[2] for m in matched_pred if abs(m[2]) < 5]
rms_yr = np.sqrt(np.mean(np.array(residuals)**2))
rms_days = rms_yr * 365.25

print(f"\nRMS residual (excluding 2022): {rms_yr:.3f} yr = {rms_days:.0f} days")

# ============================================================
# SECTION 8: CRITICAL TEST — 2022 PREDICTION
# ============================================================
print(f"\n{'='*70}")
print("SECTION 8: THE 2022 PREDICTION")
print("="*70)

# Find what WILL predicts near 2022
pred_near_2022 = [(ep, cr) for ep, cr in pred_epochs
                   if 2020 < ep < 2026]

print(f"\nWILL predictions near 2020-2026:")
for ep, cr in sorted(pred_near_2022):
    print(f"  {ep:.3f}  (orbit {cr['orbit']}, phase {np.degrees(cr['o_phase']):.1f}°, "
          f"r = {cr['r_cross']/Rs_will:.1f} R_s)")

# Valtonen GR prediction for 2022
print(f"\nValtonen (GR) predicted: 2022.70 ± few weeks")
print(f"Komossa et al. (2023):   NOT OBSERVED")

# ============================================================
# SECTION 9: FUTURE PREDICTIONS
# ============================================================
print(f"\n{'='*70}")
print("SECTION 9: WILL FUTURE PREDICTIONS")
print("="*70)

pred_future = [(ep, cr) for ep, cr in pred_epochs
               if 2025 < ep < 2045]

print(f"\nWILL predicted flare epochs (2025-2045):")
for ep, cr in sorted(pred_future):
    print(f"  {ep:.2f}  (orbit {cr['orbit']}, r = {cr['r_cross']/Rs_will:.1f} R_s, "
          f"phase = {np.degrees(cr['o_phase']):.1f}°)")

# ============================================================
# SECTION 10: SUMMARY OF ALL EQUATIONS AND VALUES
# ============================================================
print(f"\n{'='*70}")
print("SECTION 10: COMPLETE PARAMETER SUMMARY")
print("="*70)

print(f"""
INPUT OBSERVATIONAL DATA:
  T_obs        = {T_observed} yr
  e            = {e_observed}
  Δφ_obs       = {prec_observed_deg}°/orbit = {prec_obs_rad:.8f} rad

WILL-DERIVED PARAMETERS:
  κ_p²         = Δφ(1+e)/(3π) = {kp2_will:.10f}               [Eq. W10 inverted]
  κ_p          = {kp_will:.10f}
  κ²           = κ_p²(1-e)    = {kappa2_will:.10e}           [Eq. W3]
  κ            = {kappa_will:.10f}
  β            = κ/√2         = {beta_will:.10f}              [Eq. W4]
  β_p          = √(κ_p²(1+e)/2) = {bp_will:.10f}            [Eq. W7]
  Q            = √(3/2)·κ     = {Q_will:.10f}                [Eq. W5]
  W            = β²/2         = {W_will:.10e}                [Eq. W6]
  a            = Tβc/(2π)     = {a_will:.6e} m = {a_will/pc:.6f} pc  [Eq. W12]
  R_s          = κ²·a         = {Rs_will:.6e} m = {Rs_will/AU:.2f} AU    [Eq. W8]
  M            = R_s·c²/(2G)  = {M_will_solar:.4e} M☉        [Eq. W13]
  r_p          = R_s/κ_p²     = {r_p_will:.6e} m = {r_p_will/Rs_will:.1f} R_s  [Eq. W9]
  r_a          = a(1+e)       = {r_a_will:.6e} m

FIT PARAMETERS:
  t₀           = {t0_fit:.3f} (perihelion epoch)
  ω₀           = {omega0_fit:.2f}° (disk orientation)
  RMS residual = {rms_days:.0f} days
""")


╔══════════════════════════════════════════════════════════════════════╗
║        OJ 287: WILL RG ORBITAL ANALYSIS AND FLARE TIMING           ║
╠══════════════════════════════════════════════════════════════════════╣
║  All equations explicitly documented for reproducibility            ║
╚══════════════════════════════════════════════════════════════════════╝

SECTION 1: OBSERVATIONAL INPUT DATA

Observed flare epochs (decimal years):
  ✓ 1912.98  (historical)
  ✓ 1947.28  (historical)
  ✓ 1957.08  (historical)
  ✓ 1972.94  (historical)
  ✓ 1982.96  (historical)
  ✓ 1984.12  (historical)
  ✓ 1995.84  (observed)
  ✓ 2005.74  (observed)
  ✓ 2007.69  (observed)
  ✓ 2015.87  (observed (predicted to 2 weeks))
  ✓ 2019.57  (observed (predicted to 4 hours))
  ✗ 2022.70  (NOT OBSERVED (Komossa et al. 2023))

Key observational parameters:
  T_obs  = 12.06 yr (mean flare recurrence)
  e_obs  = 0.66
  Δφ_obs = 39.1°/orbit

SECTION 2: WILL RG EQUATIONS USED

FUNDAMENTAL RELATIONS:
  κ² = R_s/r   